In [3]:
import pandas as pd
from sdv import SDV
from sdv import Metadata
from sdv.tabular import GaussianCopula
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate
from sdv.tabular import TVAE
from sdv.tabular import CopulaGAN21111111111111111111111111111111111111111111111111111111111111111
import numpy as np

In [12]:
#Dataframe for NHANES survey
nhan_df = pd.read_csv('Subsample_30_data_For_SDV.csv')

In [5]:
nhan_df


,Gender,Age,Race,Birth_Country,Citizenship,Edu_Adult,Marital_Status,HH_Numb,FAM_Income_Poverty_Ratio,BMI,A1c,Health_Insurance,Measured_Diabetes_x2
0,Male,22,Black,USA,Citizen,Some_College,Never_Married,3,2.08,28.0,5.6,No,No_Risk
1,Male,20,Black,USA,Citizen,Between9-11,Never_Married,2,2.15,22.2,5.7,No,No_Risk
2,Male,42,Black,Other,Non_Citizen,College_Grad,Married,3,2.08,29.1,5.7,Yes,No_Risk
3,Male,80,Black,USA,Citizen,Below9th,Married,2,2.18,29.9,6.6,Yes,Risk
4,Female,47,Black,USA,Citizen,College_Grad,Divorced,4,0.52,48.2,6.2,Yes,Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2421,Female,20,White,USA,Citizen,Some_College,Never_Married,3,1.68,23.4,5.0,Yes,No_Risk
2422,Male,33,White,USA,Citizen,HS_Grad,Married,5,2.04,22.4,5.2,No,No_Risk
2423,Female,62,Hispanic,USA,Citizen,Some_College,Married,4,5.00,28.7,5.2,Yes,No_Risk
2424,Female,43,White,USA,Citizen,HS_Grad,Married,3,0.95,35.1,5.9,Yes,Risk


In [5]:
#Subset of NHANES data that only has black people
black_nhan = nhan_df[nhan_df['Race'] == 'Black'] 
black_nhan

,Gender,Age,Race,Birth_Country,Citizenship,Edu_Adult,Marital_Status,HH_Numb,FAM_Income_Poverty_Ratio,BMI,A1c,Health_Insurance,Measured_Diabetes_x2
0,Male,22,Black,USA,Citizen,Some_College,Never_Married,3,2.08,28.0,5.6,No,No_Risk
1,Male,20,Black,USA,Citizen,Between9-11,Never_Married,2,2.15,22.2,5.7,No,No_Risk
2,Male,42,Black,Other,Non_Citizen,College_Grad,Married,3,2.08,29.1,5.7,Yes,No_Risk
3,Male,80,Black,USA,Citizen,Below9th,Married,2,2.18,29.9,6.6,Yes,Risk
4,Female,47,Black,USA,Citizen,College_Grad,Divorced,4,0.52,48.2,6.2,Yes,Risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,Female,23,Black,USA,Citizen,Some_College,Living_with_partner,4,0.37,23.6,5.2,No,No_Risk
243,Male,62,Black,USA,Citizen,Between9-11,Married,4,1.84,34.6,5.7,Yes,No_Risk
244,Male,57,Black,USA,Citizen,HS_Grad,Never_Married,5,4.25,34.0,6.2,Yes,Risk
245,Male,80,Black,USA,Citizen,Some_College,Widowed,1,5.00,28.1,4.4,Yes,No_Risk


In [8]:
#Creates the Coplua GAN model
model = CopulaGAN(
    
    field_distributions={
        'HH_Numb': 'bounded'
        ,'A1c':'gaussian'
    }
)
model.fit(black_nhan)

In [13]:
#CTGAN version of the model
model = CTGAN()

model.fit(black_nhan)

C:\Users\Public\anaconda\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Public\anaconda\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass n_components=10 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\Public\anaconda\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\Public\anaconda\lib\site-packages\sklearn\mixture\_base.py:265: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check f

In [14]:
#Conditions to sample only black data
conditions = {
    'Race' : 'Black'
}
synth_50 = model.sample(165,conditions=conditions)

In [13]:
conditions = {
    'Race' : 'Black'
}
synth_70 = model.sample(330,conditions=conditions)

In [14]:
conditions = {
    'Race' : 'Black'
}
synth_optimal = model.sample(690,conditions=conditions)

In [15]:
#Used to recode the categorical columns
race_cond = [
    (synth_50['Edu_Adult'] == 'Below9th'),
    (synth_50['Edu_Adult'] == 'Between9-11'),
    (synth_50['Edu_Adult'] == 'HS_Grad'),
    (synth_50['Edu_Adult'] == 'Some_College'),
    (synth_50['Edu_Adult'] == 'College_Grad')
]
race_values = [1,2,3,4,5]

mari_cond = [
    (synth_50['Marital_Status'] == 'Married'),
    (synth_50['Marital_Status'] == 'Divorced'),
    (synth_50['Marital_Status'] == 'Widowed'),
    (synth_50['Marital_Status'] == 'Separated'),
    (synth_50['Marital_Status'] == 'Living_with_partner'),
    (synth_50['Marital_Status'] == 'Never_Married')
]

mari_values = [1,2,2,2,1,3]

In [16]:
#Maps numerical values for categories
synth_50['Gender_'] = [1 if x=='Male' else 2 for x in synth_50['Gender']]
synth_50['Measured_Diabetes_x2_'] = [0 if x=='No_Risk' else 1 for x in synth_50['Measured_Diabetes_x2']]
synth_50['Birth_Country_'] = [1 if x=='USA' else 2 for x in synth_50['Birth_Country']]
synth_50['Citizenship_'] = [1 if x=='Citizen' else 2 for x in synth_50['Citizenship']]
synth_50['Edu_Adult_'] = np.select(race_cond,race_values)
synth_50['Marital_Status_'] = np.select(mari_cond,mari_values)
synth_50['Race_'] = 4

In [17]:
#Drops old columns
synth_50.drop(['Gender','Birth_Country','Citizenship','Edu_Adult','Marital_Status','Race','Measured_Diabetes_x2'],axis=1,inplace=True)

In [19]:
synth_50.head()
synth_50.to_csv("CTGAN_Synthetic_Data_50.csv")

In [23]:
race_cond = [
    (synth_70['Edu_Adult'] == 'Below9th'),
    (synth_70['Edu_Adult'] == 'Between9-11'),
    (synth_70['Edu_Adult'] == 'HS_Grad'),
    (synth_70['Edu_Adult'] == 'Some_College'),
    (synth_70['Edu_Adult'] == 'College_Grad')
]
race_values = [1,2,3,4,5]

mari_cond = [
    (synth_70['Marital_Status'] == 'Married'),
    (synth_70['Marital_Status'] == 'Divorced'),
    (synth_70['Marital_Status'] == 'Widowed'),
    (synth_70['Marital_Status'] == 'Separated'),
    (synth_70['Marital_Status'] == 'Living_with_partner'),
    (synth_70['Marital_Status'] == 'Never married')
]

mari_values = [1,2,2,2,1,3]

In [24]:
synth_70['Gender_'] = [1 if x=='Male' else 2 for x in synth_70['Gender']]
synth_70['Measured_Diabetes_x2_'] = [0 if x=='No_Risk' else 1 for x in synth_70['Measured_Diabetes_x2']]
synth_70['Birth_Country_'] = [1 if x=='USA' else 2 for x in synth_70['Birth_Country']]
synth_70['Citizenship_'] = [1 if x=='Citizen' else 2 for x in synth_70['Citizenship']]
synth_70['Edu_Adult_'] = np.select(race_cond,race_values)
synth_70['Marital_Status_'] = np.select(mari_cond,mari_values)
synth_70['Race_'] = 4

In [25]:

synth_70.drop(['Gender','Birth_Country','Citizenship','Edu_Adult','Marital_Status','Race','Measured_Diabetes_x2'],axis=1,inplace=True)

In [26]:
synth_70.head()
synth_70.to_csv("New_Synthetic_Data_70.csv")

In [20]:
new_data.Health_Insurance.value_counts()

1    177
2     23
Name: Health_Insurance, dtype: int64

In [26]:
new_data.Race.value_counts()

3    714
7    213
4     62
6      9
1      2
Name: Race, dtype: int64

In [16]:
new_data.HH_Numb.value_counts()

2    754
3    365
1    306
4    261
5    128
6     96
7     90
Name: HH_Numb, dtype: int64

In [35]:
nhan_df.Gender.value_counts()

2    1987
1    1946
Name: Gender, dtype: int64